In [2]:
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from polygon import RESTClient
from datetime import datetime, timedelta
import pandas_market_calendars as mcal
import polygon
import os, pickle

In [149]:
datetime_diff = lambda date1, date2 : (datetime.strptime(date1, '%Y-%m-%d') - datetime.strptime(date2, '%Y-%m-%d')).days

### Data Collection

In [3]:
load_dotenv("/Users/brad/mlprojects/guidelight/guidelight-api/.env")
token = os.getenv("POLYGON_TOKEN")

In [4]:
client = RESTClient(api_key=token)

In [5]:
import pickle
# pickle.dump(data, open("options_data/AAPL-2023-03-27.pkl", "wb"))
if not os.path.exists("all_contracts.pkl"):
	reqs = client.list_options_contracts("AAPL",as_of="2024-04-16", expired=True, expiration_date_gt="2023-04-16")
	all_contracts = list(reqs)
	pickle.dump(all_contracts, open("all_contracts.pkl", "wb"))
else:
	all_contracts = pickle.load(open("all_contracts.pkl", "rb"))

In [49]:
len(all_contracts)

6164

In [43]:
import pandas as pd

# data = {
#     ('AAPL_2207C00500000', '2023-07-21'): [
#         {'trading_date': '2023-07-01', 'strike': 500, 'open': 15.5, 'high': 16.0, 'low': 15.0, 'close': 15.75, 'volume': 300},
#         {'trading_date': '2023-07-02', 'strike': 500, 'open': 15.6, 'high': 17.0, 'low': 15.2, 'close': 16.50, 'volume': 350}
#     ],
#     ('MSFT_2207P00250000', '2023-07-21'): [
#         {'trading_date': '2023-07-01', 'strike': 250, 'open': 8.5, 'high': 9.0, 'low': 8.0, 'close': 8.75, 'volume': 200},
#         {'trading_date': '2023-07-02', 'strike': 250, 'open': 8.6, 'high': 10.0, 'low': 8.1, 'close': 9.75, 'volume': 250}
#     ]
# }
if not os.path.exists("data.pkl"):
    indices = [(contract.ticker, contract.expiration_date, contract.strike_price) for contract in all_contracts]
    data = {}
    for index in indices:
        ticker, expiration_date, strike_price = index
        current_date = datetime.strptime(expiration_date, "%Y-%m-%d")
        past_date = current_date - timedelta(days=14)

        # get key value data for each agg

        a = [vars(agg) for agg in client.get_aggs(ticker, 1, 'day', past_date, current_date)]
        data[index] = a


    pickle.dump(data, open("data.pkl", "wb"))

else:
    data = pickle.load(open("data.pkl", "rb"))

In [ ]:

# dataset = pd.DataFrame(columns=['Weight', 'price', 'maturity', 'S'])
# dataset['Weight'] = df['volume'] / df.volume.sum()
# dataset['price']  = df['price']
# dataset['maturity'] = df['maturity']
# dataset['S']        = df['strike_price']
# dataset['time_from_last_trade'] = -df['days since last trade']
# dataset

In [47]:
# Flatten the data while preserving the option ticker and expiration date
flattened_data = []
for (ticker, expiration, strike_price), entries in data.items():
    for entry in entries:
        entry.update({
            "ticker": ticker,
            "expiration_date": expiration,
            "strike_price": strike_price
        })
        flattened_data.append(entry)

# Create a DataFrame
df = pd.DataFrame(flattened_data)

# Set a MultiIndex using the ticker, expiration date, and trading date
df.set_index(['ticker', "strike_price", 'expiration_date'], inplace=True)

# get by ticker
# 1681099200000
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.strftime("%Y-%m-%d")
# df.index = df.index.set_levels(pd.to_datetime(df.index.get_level_values('timestamp'), unit='ms').strftime('%Y-%m-%d %H:%M:%S'), level='timestamp')

In [64]:
df.to_csv("options_data-aapl.csv", index_label=['ticker', "strike_price", 'expiration_date'])

In [153]:
df2 = pd.read_csv('options_data-aapl.csv', index_col=[0, 1, 2])
# get only the tickers traded on 2023-04-10
# df2.groupby('timestamp').filter(lambda x: x['timestamp']== '2023-04-10')
# get the level values

agg_series= df.loc[('O:AAPL230421C00050000', slice(None), slice(None))]
target_timestamp = '2023-04-19'

# Create a boolean mask for rows where the timestamp matches the target
i = np.where(agg_series['timestamp'].values == target_timestamp)[0][0]
# agg_series
diff = datetime_diff(agg_series['timestamp'].iloc[i], agg_series['timestamp'].iloc[i-1])
diff

9

In [ ]:
def daily_option_data(underlying_ticker:str, date:str):
	# client = RESTClient(api_key=token)
	df = pd.read_csv(f'options_data-{underlying_ticker}.csv', index_col=[0, 1, 2])
	option_contracts = df2.loc[df2['timestamp'] == date]
	colnames = ["ticker", "maturity", "Weight", 'volume', 'price', 'days since last trade', 'S']
	df = pd.DataFrame(columns=colnames)

	for ticker in option_contracts.get_level_values(0).unique():
		agg_series= df.loc[(ticker, slice(None), slice(None))]
		
		i = np.where(agg_series['timestamp'].values == date)[0][0]
	# agg_series
		diff = datetime_diff(agg_series['timestamp'].iloc[i], agg_series['timestamp'].iloc[i-1])

		a = pd.DataFrame([contract.__dict__.values()], columns=cols)
		a['maturity'] = (expiration_time - today).days / 365
		a['volume'] = option_contracts[i]["volume"]
		a['price'] = option_contracts[i]["vwap"]
		a['days since last trade'] = diff
		df = pd.concat([df, a])

	
	df.to_csv( os.path.join(os.getcwd(), f"options_data/{underlying_ticker}-{date}.csv"))

In [ ]:
import pandas as pd
from multiprocessing import Pool
import os

def worker(date, underlying_ticker, contract_type):
    load_dotenv("/Users/brad/mlprojects/guidelight/guidelight-api/.env")
    token = os.getenv("POLYGON_TOKEN")
    return daily_option_data(underlying_ticker, date, token, contract_type)


def process_multiple_days(underlying_ticker, start_date, end_date, contract_type="put"):
    # Generate list of dates
    dates = mcal.get_calendar("NYSE").valid_days(start_date=start_date, end_date=end_date)
    
    # Define a helper to wrap your existing function for use with starmap
    

    # Setup multiprocessing pool
    with Pool() as pool:
        pool.starmap(worker, [(date, underlying_ticker, contract_type) for date in dates])

    print("Data processing complete for all specified dates.")




process_multiple_days('AAPL', '2023-04-17', '2024-04-16')

In [ ]:
yield_rates = pd.read_csv("five-year-rates.csv")
yield_rates

In [ ]:
ticker = "AAPL"

# List Aggregates (Bars)
aggs = []
for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_="2022-01-01", to="2023-12-30", limit=50000, options=T):
    aggs.append(a)

print(len(aggs))



In [ ]:
aggs[-1]

In [ ]:
#unix millisecond timestamp to datetime
# from pytz import timezone

# Convert timestamp to datetime in UTC timezone
def ts_to_dt(timestamp):
    return datetime.fromtimestamp(timestamp/1000, tz=timezone.utc)

timestamps = np.vectorize(ts_to_dt)

# Example usage
a = np.array([a.timestamp for a in aggs])
times = timestamps(a)
aapl_aggs = pd.DataFrame({"day": [time.strftime("%Y-%m-%d") for time in times],
                            "timestamp": [time.strftime("%Y-%m-%d, %H:%M") for time in times],
                           "close": [a.close for a in aggs], 
                           "volume": [a.volume for a in aggs],
                           "low": [a.low for a in aggs],
                           "high": [a.high for a in aggs]})



In [ ]:
from heston_param import *

In [ ]:
aapl_aggs.groupby("day")

In [ ]:
daily_aggs =  [aapl_aggs.groupby('day').get_group(x) for x in aapl_aggs['day'].unique()]

In [ ]:
hist_voilatilities = torch.empty(len(daily_aggs))
for i, day in enumerate(daily_aggs):
	hist_voilatilities[i] = estimate_historical_volatility(day['close'].values)

# calculate the historical volatility for each day





In [ ]:
import pickle
from heston_param import *
hist_voilatilities = pickle.load(open("hist_voilatilities.pkl", "rb"))
daily_aggs = pickle.load(open("daily_aggs.pkl", "rb"))

In [ ]:
while True:
	try:
		print(calibrate_daily_parameters(hist_voilatilities[0], 0.1, daily_aggs[0]["close"].values, 0.0237, daily_aggs[0]["close"].values.shape[0], 50))
		break
	except RuntimeError:
		continue

In [ ]:
def single_day_calibration(args):
		i, hist_volatilities, daily_agg = args
		while True:
			try:
				params = calibrate_daily_parameters(hist_volatilities[i], 0.1, day["close"].values, 0.0237, day["close"].values.shape[0], 300)
				
				return i, params
			except RuntimeError:
				continue

In [ ]:
len(daily_aggs)

### Naive Monte Carlo:
Runtime: 1.15 Hour. Suboptimal.

In [ ]:
daily_params = np.empty((len(daily_aggs), 5))
for i, day in enumerate((daily_aggs[:20])):
	while True:
		try:
			daily_params[i] = calibrate_daily_parameters(hist_voilatilities[i], 0.1, daily_aggs[i]["close"].values, 0.0237, daily_aggs[i]["close"].values.shape[0], 50)
			if (i + 1) % 10 == 0:
					rate =  100 *  np.round((i + 1) /len(daily_params[:20]), 2)
					print(f"{rate}% completed.")
			
			break
		except RuntimeError:
			continue



In [ ]:
def calibrate_worker(args):
	single_day_calibration(args)

In [ ]:
daily_params = pickle.load(open("parameters.pkl", "rb"))

In [ ]:
# runtime is 1.5 hours
daily_params.shape

In [ ]:
from multiprocess import Pool

def calibrate_parameters_multiprocessing(daily_aggs, hist_volatilities):
    # Prepare arguments for each task
    tasks = [(i, hist_volatilities[i], daily_aggs[i]) for i in range(len(daily_aggs))]

    # Number of processes, could be set to the number of CPUs or cores
    num_processes = 4

    # Create a multiprocessing pool and map tasks to worker function
    with Pool(processes=num_processes) as pool:
        results = pool.map(calibrate_worker, tasks)

    # Process results
    daily_params = np.empty((len(daily_aggs[:20]), 5))
    for index, params in results:
        if params is not None:
            daily_params[index] = params
        else:
            print(f"Calibration failed for index {index}")

    return daily_params



In [ ]:
calibrate_parameters_multiprocessing(daily_aggs[:20], hist_voilatilities[:20])

https://medium.com/@alexander.tsoskounoglou/pricing-options-with-fourier-series-p3-the-heston-model-d157369a217a

In [ ]:
def heston_char(u, params):
    kappa, theta, zeta, rho, v0, r, q, T, S0 = params 
    t0 = 0.0 ;  q = 0.0
    m = np.log(S0) + (r - q)*(T-t0)
    D = np.sqrt((rho*zeta*1j*u - kappa)**2 + zeta**2*(1j*u + u**2))
    C = (kappa - rho*zeta*1j*u - D) / (kappa - rho*zeta*1j*u + D)
    beta = ((kappa - rho*zeta*1j*u - D)*(1-np.exp(-D*(T-t0)))) / (zeta**2*(1-C*np.exp(-D*(T-t0))))
    alpha = ((kappa*theta)/(zeta**2))*((kappa - rho*zeta*1j*u - D)*(T-t0) - 2*np.log((1-C*np.exp(-D*(T-t0))/(1-C))))
    return np.exp(1j*u*m + alpha + beta*v0)

In [ ]:
import numpy as np
from numpy import sqrt, exp, pi, cos, sin, log, abs
from numba import njit

@njit
def Fourier_Heston_Put(S0, K, T, r, 
                  # Heston Model Paramters
                  kappa, # Speed of the mean reversion 
                  theta, # Long term mean
                  rho,   # correlation between 2 random variables
                  zeta,  # Volatility of volatility
                  v0,    # Initial volatility 
                  opt_type,
                  N = 1_012,
                  z = 24
                  ):

  def heston_char(u): 
    t0 = 0.0 ;  q = 0.0
    m = log(S0) + (r - q)*(T-t0)
    D = sqrt((rho*zeta*1j*u - kappa)**2 + zeta**2*(1j*u + u**2))
    C = (kappa - rho*zeta*1j*u - D) / (kappa - rho*zeta*1j*u + D)
    beta = ((kappa - rho*zeta*1j*u - D)*(1-exp(-D*(T-t0)))) / (zeta**2*(1-C*exp(-D*(T-t0))))
    alpha = ((kappa*theta)/(zeta**2))*((kappa - rho*zeta*1j*u - D)*(T-t0) - 2*log((1-C*exp(-D*(T-t0))/(1-C))))
    return exp(1j*u*m + alpha + beta*v0)
  
  # # Parameters for the Function to make sure the approximations are correct.
  c1 = log(S0) + r*T - .5*theta*T
  c2 = theta/(8*kappa**3)*(-zeta**2*exp(-2*kappa*T) + 4*zeta*exp(-kappa*T)*(zeta-2*kappa*rho) 
        + 2*kappa*T*(4*kappa**2 + zeta**2 - 4*kappa*zeta*rho) + zeta*(8*kappa*rho - 3*zeta))
  a = c1 - z*sqrt(abs(c2))
  b = c1 + z*sqrt(abs(c2))
  
  h       = lambda n : (n*pi) / (b-a) 
  g_n     = lambda n : (exp(a) - (K/h(n))*sin(h(n)*(a - log(K))) - K*cos(h(n)*(a - log(K)))) / (1 + h(n)**2)
  g0      = K*(log(K) - a - 1) + exp(a)
  
  F = g0 
  for n in range(1, N+1):
    h_n = h(n)
    F += 2*heston_char(h_n) * exp(-1j*a*h_n) * g_n(n)

  F = exp(-r*T)/(b-a) * np.real(F)
  F = F if opt_type == 'p' else F + S0 - K*exp(-r*T)
  return F if F > 0 else 0




In [ ]:
S0      = 100.      # initial asset price
K       = 50.       # strike
r       = 0.03      # risk free rate
T       = 1/365     # time to maturity

v0=0.4173 ; kappa=0.4352 ; theta=0.2982 ; zeta=1.3856 ; rho=-0.0304

In [ ]:
import py_vollib_vectorized

price = 0.10 ; S = 95 ; K = 100 ; t = .2 ; r = .2 ; flag = 'c'

def implied_volatility(price, S, K, t, r, flag):
  return py_vollib_vectorized.vectorized_implied_volatility(
    price, S, K, t, r, flag, q=0.0, on_error='ignore', model='black_scholes_merton',return_as='numpy') 


In [ ]:
print('Speed Analysis of the implied volatility Function: Per Option')
%timeit implied_volatility(price, S, K, t, r, flag)

In [ ]:
# %pip install pyFFTW
import pyfftw

In [ ]:
import numpy as np
from numpy import exp, pi, log, sqrt
# from  numpy.fft import fft
import pyfftw
# @njit
def heston_fft2(S0, K, T, r, kappa, theta, rho, zeta, v0, opt_type, N=1024, alpha=1.5):
    eta = 0.25  # Grid spacing for the integration variable
    lambda_u = 2 * pi / (N * eta)  # Grid spacing for log strike

    # Adjustments for the damping factor
    alpha = alpha  # Damping factor, typically 1 or 1.5

    # Characteristic function for the Heston model as before
    def heston_char(u):
        t0 = 0.0 ; q = 0.0
        m = log(S0) + (r - q) * (T - t0)
        D = sqrt((rho * zeta * 1j * u - kappa) ** 2 + zeta ** 2 * (1j * u + u ** 2))
        C = (kappa - rho * zeta * 1j * u - D) / (kappa - rho * zeta * 1j * u + D)
        beta = ((kappa - rho * zeta * 1j * u - D) * (1 - exp(-D * (T - t0)))) / (zeta ** 2 * (1 - C * exp(-D * (T - t0))))
        alpha = ((kappa * theta) / (zeta ** 2)) * ((kappa - rho * zeta * 1j * u - D) * (T - t0) - 2 * log((1 - C * exp(-D * (T - t0))) / (1 - C)))
        return exp(1j * u * m + alpha + beta * v0)

    # Array of discretized u values (integration variable)
    u = np.arange(1, N) * eta
    u = np.hstack((np.array([0.000001]), u))  # Avoid division by zero in calculations

    # Weights for the integration
    weights = np.ones(N)
    weights[0] = 0.5  # Trapezoidal rule: first weight is 0.5
    weights = weights * eta

    # Damping factor applied to characteristic function
    adjusted_char_fn = exp(-r * T) * (heston_char(u - (alpha + 1) * 1j) / (alpha ** 2 + alpha - u ** 2 + 1j * (2 * alpha + 1) * u))

    # FFT calculation
    fft_object = pyfftw.builders.rfft(adjusted_char_fn.real * weights, threads=4)
    fft_values = fft_object()
    fft_values = fft_values[:N // 2]  # Only need the first half of the FFT output

    # Calculate strike prices corresponding to FFT output
    strikes = S0 * exp(-lambda_u * np.arange(N // 2))

    # Option prices
    prices = np.exp(-alpha * np.log(strikes)) / pi * fft_values.real

    # Find the index of the strike closest to K
    index = np.argmin(np.abs(strikes - K))
    price = prices[index]
    return price if opt_type == 'p' else price + S0 - K * exp(-r * T)

# Example call to function
price = heston_fft2(S0=100, K=100, T=1, r=0.05, kappa=0.2, theta=0.04, rho=-0.7, zeta=0.2, v0=0.04, opt_type='p')
                   


In [ ]:
price

In [ ]:
Fourier_Heston_Put(S0=100, K=100, T=1, r=0.05, kappa=0.2, theta=0.04, rho=-0.7, zeta=0.2, v0=0.04, opt_type='p')

In [ ]:
from nelson_siegel_svensson.calibrate import calibrate_nss_ols
yield_maturities = np.array([1/12, 2/12, 3/12, 4/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])
# yields  = np.array([5.30,5.39,5.50,5.50,5.44,5.11,4.33,3.98,3.70,3.66,3.61,3.98,3.84])
# get the first row of the yield rates
yields = yield_rates.iloc[0].values[1:].astype(np.float64)
curve_fit, status = calibrate_nss_ols(yield_maturities,yields)
# yields

In [ ]:
curve_fit, status

In [ ]:
print('Speed Analysis of Fourier_Heston_Put: Per Option')
%timeit Fourier_Heston_Put(S0,K, T, r, kappa, theta, rho, zeta, v0, 'p', N = 1_012, z = 24)

In [ ]:
def get_implied_volatility(price, S, K, t, r, flag):
    return py_vollib_vectorized.vectorized_implied_volatility(
        price, S, K, t, r, flag, q=0.0, on_error='ignore', model='black_scholes_merton',return_as='numpy') 

In [ ]:
volSurfaceLong = (
    df[
        (df['maturity'] > 0)
        & (df['DaysSinceLastTraded'] >= Lookback_limit)
    ]
)
volSurfaceLong['rate'] = volSurfaceLong['maturity'].apply(curve_fit) / 100
volSurfaceLong         = volSurfaceLong.reset_index()

# Data Cleaning: Sets the inf weighted values to the maximum weight 
volSurfaceLong.loc[ volSurfaceLong['Weight'] == np.inf , 'Weight'] = volSurfaceLong[ volSurfaceLong['Weight'] != np.inf ]['Weight'].max()
volSurfaceLong['Weight'] = volSurfaceLong['Weight'] / volSurfaceLong['Weight'].mean()

volSurface = volSurfaceLong.sample(n=250)
_K          = volSurface['strike'].to_numpy()
_P          = volSurface['price'].to_numpy()
_T          = volSurface['maturity'].to_numpy()
_r          = volSurface['rate'].to_numpy()
_S          = volSurface['S'].to_numpy()
opt_types   = volSurface['Type'].to_numpy(dtype=str)

volSurface['IV'] = get_implied_volatility(price=_P, S=_S, K=_K, t=_T, r=_r, flag=opt_types)
volSurface = volSurface[~volSurface.IV.isna()]
print(f'Options Droped where IV == 0: {(~(volSurface.IV > 0)).sum()} , aka Options which the middpoint cannot be priced')
volSurface = volSurface[volSurface.IV > 0]
volSurface = volSurface[(volSurface['IV'] != np.inf) & (volSurface['IV'] != np.nan)]

In [ ]:
Lookback_limit = dataset['DaysSinceLastTraded'].sort_values().unique()[-1]
volSurfaceLong = (
    df[
        (df['maturity'] > 0)
        & (df['DaysSinceLastTraded'] >= Lookback_limit)
    ]
)


In [ ]:
def get_error_Heston(volSurface, v0, kappa, theta, zeta, rho):
    """Calculates the error between the Heston model and the market prices.
    Arguments:
        volSurface: DataFrame with the market prices.
        v0: Initial variance.
        kappa: Mean reversion speed.
        theta: Long-run variance.
        zeta: Volatility of volatility.
        rho: Correlation between the variance and the asset.
    """
    error = 0
    for _, row in volSurface.iterrows():
        P = row['price']
        HP = Fourier_Heston_Put(S0=row['S'], K=row['strike'], v0=v0, kappa=kappa, theta=theta, zeta=zeta, rho=rho, T=row['maturity'], r=row['rate'], N=2048)
        error += (P - HP)**2

    return error / volSurface.shape[0]

def get_resutls_array_Heston(volSurface, v0, kappa, theta, zeta, rho, N=10_000, z=64):
    # Initialize the results array
    results = -np.ones(volSurface.shape[0])
    # reset the index of the options dataframe
    volSurface.index = np.arange(0, volSurface.shape[0])
    # loop through the rows of the options dataframe and run the Fourier_Heston_Put function
    for idx, row in volSurface.iterrows():
        results[idx] = Fourier_Heston_Put(S0=int(row['S']), K=int(row['strike']), v0=v0, kappa=kappa, theta=theta, zeta=zeta, rho=rho, T=row['maturity'], r=row['rate'], N=N, opt_type=row['Type'],z=z)
    return results

def get_resutls_df_Heston(volSurface, v0, kappa, theta, zeta, rho, N=2048, z=100):
    observed = volSurface.copy(deep=True)
    heston = volSurface.copy(deep=True)
    observed['source'] = 'Observed'
    heston['source'] = 'Heston Model'

    heston_prices = [] 
    implied_volatilities = []
    for _, row in volSurface.iterrows():
        heston_price = Fourier_Heston_Put(S0=row['S'], K=row['strike'], v0=v0, kappa=kappa, theta=theta, zeta=zeta, rho=rho, T=row['maturity'], r=row['rate'], N=N, opt_type=row['Type'], z=z)
        heston_prices.append(heston_price)
        # np.array(... , ndmin=1) So the type of the input is compatible with what numba expects
        maturity  = np.array(row['maturity'],ndmin=1)
        observed_price  = np.array(heston_price,ndmin=1)
        S0 = np.array(row['S'],ndmin=1)
        K  = np.array(row['strike'],ndmin=1)
        r  = np.array(row['rate'],ndmin=1)
        option_type = np.array(row['Type'],ndmin=1)
        implied_volatility = get_implied_volatility(price=observed_price, S=S0, K=K, t=maturity, r=r, flag=option_type)
        implied_volatilities.append(implied_volatility[0])

    heston['price'] = heston_prices
    heston['IV']    = implied_volatilities

    return pd.concat([observed, heston])

def get_error_df_Heston(volSurface, v0, kappa, theta, zeta, rho, diff='Price', error='Error', weighted=True, N=10_000, z=64):
    if   error == 'Error':          _name = f'Weighted Error {diff}'             if weighted else f'Error {diff}'
    elif error == 'Perc Error':     _name = f'Weighted Persentage Error {diff}'  if weighted else f'Persentage Error {diff}'
    elif error == 'Squared Error':  _name = f'Weighted Squared Error {diff}'     if weighted else f'Squared Error {diff}'
    else: raise Exception("Error: variable 'error' is not defined correctly")
    
    results_df = {'strike':[], 'maturity':[], _name:[], 'Opt. Type':[], 'Weight':[]}

    for _, row in volSurface.copy(deep=True).iterrows():
        _P = Fourier_Heston_Put(S0=row['S'], K=row['strike'], v0=v0, kappa=kappa, theta=theta, zeta=zeta, rho=rho, T=row['maturity'], r=row['rate'], N=N, z=z, opt_type=row['Type'])
        # np.array(... , ndmin=1) So the type of the input is compatible with what numba expects
        _T  = np.array(row['maturity'],ndmin=1)
        _C  = np.array(_P,ndmin=1)
        _P  = np.array(row['price'],ndmin=1)
        _S0 = np.array(row['S'],ndmin=1)
        _K  = np.array(row['strike'],ndmin=1)
        _r  = np.array(row['rate'],ndmin=1)
        _OT = np.array(row['Type'],ndmin=1)

        _IV  = get_implied_volatility(price=_C, S=_S0, K=_K, t=_T, r=_r, flag=_OT)
        _IV2 = get_implied_volatility(price=row['price'], S=_S0, K=_K, t=_T, r=_r, flag=_OT)

        if error    == 'Error':
            if diff == 'IV':  _error  = (_IV - _IV2) *                (row['Weight'] if weighted else 1)
            else           :  _error  = (_C - _P) *                   (row['Weight'] if weighted else 1)
        elif error  == 'Perc Error':
            if diff == 'IV':  _error  = ((_IV - _IV2)/_IV2) * 100 *   (row['Weight'] if weighted else 1)
            else           :  _error  = ((_C - _P)/_P) * 100 *        (row['Weight'] if weighted else 1)
        elif error  == 'Squared Error':
            if diff == 'IV':  _error  = (_IV - _IV2)**2 *             (row['Weight'] if weighted else 1)
            else           :  _error  = (_C - _P)**2 *                (row['Weight'] if weighted else 1)

        results_df[_name].append(_error[0])
        results_df['maturity'].append(_T[0])
        results_df['strike'].append(_K[0])
        results_df['Opt. Type'].append(_OT[0])
        results_df['Weight'].append(row['Weight']*10)

    return pd.DataFrame(results_df)

In [ ]:
# Extract data from volSurface DataFrame
_K      = volSurface['strike'].to_numpy()
_C      = volSurface['price'].to_numpy()
_type   = volSurface['Type'].to_numpy(dtype=str)
_T      = volSurface['maturity'].to_numpy()
_r      = volSurface['rate'].to_numpy()
_S      = np.full_like(_K, S0)
_IV     = volSurface['IV'].to_numpy()
_Weight = volSurface['Weight'].to_numpy()

# Initial parameters and bounds for optimization
params = {
    "v0": {"x0": 0.002874, "lbub": [1e-3, 1.2]},
    "kappa": {"x0": 1.6891, "lbub": [1e-3, 10]},
    "theta": {"x0": 0.0190, "lbub": [1e-3, 1.2]},
    "zeta": {"x0": 3.7472, "lbub": [1e-2, 4]},
    "rho": {"x0": -0.2731, "lbub": [-1, 1]}
}
x0 = [param["x0"] for _, param in params.items()]
bnds = [param["lbub"] for _, param in params.items()]

# Objective function: Squared Error
def SqErr(x):
    v0, kappa, theta, zeta, rho = x
    
    # Calculate prices using Heston Model
    Price_Heston = get_resutls_array_Heston(
        volSurface, v0, kappa, theta, zeta, rho, N=1_012, z=24
    )
    
    # Calculate implied volatilities
    IV_Heston = get_implied_volatility(
        price=Price_Heston, S=_S, K=_K, t=_T, r=_r, flag=_type
    )
    
    # Handle undefined IV calculations
    diff = IV_Heston - _IV
    idx = np.isnan(diff) | np.isinf(diff)
    diff[idx] = 0 - _IV[idx]
    IV_Heston[idx] = 0
    
    # Calculate RMSE
    rmse = sqrt(np.mean((diff * 100) ** 2 * _Weight))
    
    # Debugging info
    zeros = int(np.where(IV_Heston == 0, 1, 0).sum())
    wmae  = np.mean(np.abs(diff * 100) * _Weight)
    print_debug_info(v0, kappa, theta, zeta, rho, wmae, idx, zeros, rmse)
    
    return rmse

# Debugging info printer
def print_debug_info(v0, kappa, theta, zeta, rho, wmae, idx, zeros, rmse):
    print(
        f">>v0={v0:.4f}; kappa={kappa:.4f}; theta={theta:.4f}; "
        f"zeta={zeta:.4f}; rho={rho:7.4f} | WMAE(IV): {wmae:.5e} | "
        f"Nulls: {idx.sum()}/{idx.shape[0]} | Zeros: {zeros}/{idx.shape[0]} | "
        f"WRMSE(IV): {rmse:.5e}"
    )

# Run optimization
result = minimize(
    SqErr, x0, tol=1e-5, method='SLSQP',
    options={'maxiter': 80, 'ftol': 1e-5, 'disp': True},
    bounds=bnds, jac='3-point'
)

# Extract optimized parameters
v0, kappa, theta, zeta, rho = result.x

In [ ]:

def heston_daily_volSurface(date, underlying_ticker):
    # Load the data
    df = pd.read_csv(f"options_data/contracts-{underlying_ticker}.csv")

    def filter_by_time_diff(x):
        current_date = datetime.strptime(date, "%Y-%m-%d")
        time_diff = datetime.strptime(x['expiration_date'], "%Y-%m-%d").days -current_date.days
        return time_diff >= 0 and time_diff < 14
    
    contracts = df.groupby('expiration_date').filter(filter_by_time_diff)
    
    # for each contract, pick the aggregates

    contract_aggs = contracts.groupby(['expiration_date', 'trading_date']).filter(lambda x: x['trading_date'] == date)
    
    # 

    return volSurface

def heston_daily_parameters(dailyVolSurface):
	# Extract data from dailyVolSurface DataFrame
    _K = dailyVolSurface['strike'].to_numpy()

    _C = dailyVolSurface['price'].to_numpy()
    _type   = dailyVolSurface['Type'].to_numpy(dtype=str)
    _T      = dailyVolSurface['maturity'].to_numpy()
    _r      = dailyVolSurface['rate'].to_numpy()
    _S      = np.full_like(_K, S0)
    _IV     = dailyVolSurface['IV'].to_numpy()
    _Weight = dailyVolSurface['Weight'].to_numpy()

    # Initial parameters and bounds for optimization
    params = {
        "v0": {"x0": 0.002874, "lbub": [1e-3, 1.2]},
        "kappa": {"x0": 1.6891, "lbub": [1e-3, 10]},
        "theta": {"x0": 0.0190, "lbub": [1e-3, 1.2]},
        "zeta": {"x0": 3.7472, "lbub": [1e-2, 4]},
        "rho": {"x0": -0.2731, "lbub": [-1, 1]}
    }
    x0 = [param["x0"] for _, param in params.items()]
    bnds = [param["lbub"] for _, param in params.items()]
    result = minimize(
    SqErr, x0, tol=1e-5, method='SLSQP',
    options={'maxiter': 80, 'ftol': 1e-5, 'disp': True},
    bounds=bnds, jac='3-point'
	)